In [21]:
import numpy as np
import random
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import os
from alpaca_trade_api.rest import REST, TimeFrame
import hvplot.pandas
from scipy.interpolate import interp1d
import models as model

import warnings
warnings.filterwarnings('ignore')

In [22]:
load_dotenv()

True

In [23]:
stock_list = ['NVDA']
df = fetch_stock_data('2018-01-12', '2024-03-14', stock_list)
df

,NVDA
timestamp,
2018-01-12,222.759
2018-01-16,220.110
2018-01-17,224.720
2018-01-18,224.440
2018-01-19,230.110
...,...
2024-03-08,875.280
2024-03-11,857.740
2024-03-12,919.130
